<a href="https://colab.research.google.com/github/hassssan051/portrait-video-synthesis/blob/audio-to-descriptor-pred/prediction/NN_next_frame_predictor_hierarchical_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import zipfile
from tqdm import tqdm
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler

import torch.nn.functional as F

In [ ]:
#loading MFCC features of RAVDESS dataset
datasetPath = 'DatasetForLSTM'
Zipped_inside_folder = 'RAVDESS_MFCC'
with zipfile.ZipFile('drive/MyDrive/'+Zipped_inside_folder+'.zip', 'r') as zip_ref:
    zip_ref.extractall(datasetPath)


In [ ]:
from collections import defaultdict
import pickle

#getting descriptors of frames of each video and storing this information in a dictionary where key is video name and value is a list of descriptors of its frames.


clusters_info = 'Sawaiz_2/pkl_for_lstm_encoded/17_53_50_800'
file_path = '/content/drive/MyDrive/'+clusters_info+'/live_portrait_descriptors_all_encoder.pkl'

# Open the file in binary read mode and load the data
with open(file_path, 'rb') as file:
    data = pickle.load(file)

video_dict = defaultdict(list)


# Populate the video_dict with frame arrays in order
for key, value in data.items():
    # Split the key to extract video name and frame number
    parts = key.split('/')
    if 'M' not in key: #For Ravdess data
      video_name = parts[1]  # Extracts '02-01-01-01-02-02-16'
      frame_number = int(parts[2].split('.')[0])  # Extracts frame number as an integer (e.g., 1)

    else: #for MEAD
      video_name = parts[0] + "__" + parts[2] + "__" + parts[3] + "__" + parts[4]
      frame_number = int(parts[-1].split(".")[0].split("_")[-1])
    # Append the frame array to the respective video entry in the dictionary
    video_dict[video_name].append((frame_number, value))

# Sort frames for each video by frame number and concatenate them into a single array
final_video_dict = {}
for video_name, frames in video_dict.items():
    # Sort frames by frame number to ensure the order is correct
    sorted_frames = sorted(frames, key=lambda x: x[0])
    # Extract only the frame data, discarding the frame numbers
    sorted_arrays = [frame_data for _, frame_data in sorted_frames]
    # Concatenate all frames into a single numpy array
    final_video_dict[video_name] = np.vstack(sorted_arrays)
true_descriptors = final_video_dict
videos_list = list(true_descriptors.keys())
print(len(videos_list))

9282


In [ ]:
# All descriptors of all frames

In [ ]:
file_path = '/content/drive/MyDrive/'+clusters_info+'/sub_cluster_average_vectors.pkl'

with open(file_path, 'rb') as file:
    clusters_data = pickle.load(file)

# Actual labels for the LP
file_path = '/content/drive/MyDrive/'+clusters_info+'/hierarchical_clusters_frame_to_cluster.pkl'
with open(file_path, 'rb') as file:
    frames_data_raw = pickle.load(file)

# Setting Indices as clusterIDs

In [ ]:
print(clusters_data.keys())

Buffered data was truncated after reaching the output size limit.

In [ ]:
# Output of heirarchical
# Frame to cluster

# I need cluster of each frame in each video in the format: Key video name and the value is the list of cluster id
# Tree clusters
#

In [ ]:
cluster_level = 4
frames_to_clusters_indices = {}
clusters_indices= {}
# Populate the video_dict with frame arrays in order
for key, value in frames_data_raw.items():
    # Split the key to extract video name and frame number
    parts = key.split('/')
    if 'M' not in key: #For Ravdess data
      video_name = parts[1]  # Extracts '02-01-01-01-02-02-16'
      frame_number = int(parts[2].split('.')[0])  # Extracts frame number as an integer (e.g., 1)

    else: #for MEAD
      video_name = parts[0] + "__" + parts[2] + "__" + parts[3] + "__" + parts[4]
      frame_number = int(parts[-1].split(".")[0].split("_")[-1])
    # Append the frame array to the respective video entry in the dictionary
    if video_name not in frames_to_clusters_indices:
      frames_to_clusters_indices[video_name] = []
    try:
      frames_to_clusters_indices[video_name].append((frame_number, value[cluster_level-1]))
      clusters_indices[value[cluster_level-1]]=0
    except:
      frames_to_clusters_indices[video_name].append((frame_number, value[-1]))
      clusters_indices[value[-1]]=0


clusters_descriptors = []
idx = 0
for key, val in clusters_indices.items():
  clusters_indices[key] = idx
  a = clusters_data[key]
  clusters_descriptors.append(clusters_data[key])
  idx+=1
clusters_descriptors = np.vstack(clusters_descriptors)

# Sort frames for each video by frame number and concatenate them into a single array
frames_data = {}
for video_name, frames in frames_to_clusters_indices.items():
    # Sort frames by frame number to ensure the order is correct
    sorted_frames = sorted(frames, key=lambda x: x[0])
    # Extract only the frame data, discarding the frame numbers
    sorted_arrays = [clusters_indices[frame_data] for _, frame_data in sorted_frames]
    # Concatenate all frames into a single numpy array
    frames_data[video_name] = sorted_arrays
#Here, frames_data is a dictionary where key is video name and value is list of cluster ids of its frames.


KeyError: 'cluster_2.cluster_2.cluster_1.cluster_1'

In [ ]:
#Here, key is a video name and value is a list of cluster representatives of those clusters to which its frames are mapped

clusters_rep_as_ground_truth_for_a_video = {}
for video, frames in frames_data.items():
  stacked_clusters_rep = [clusters_descriptors[val] for val in frames]
  clusters_rep_as_ground_truth_for_a_video[video] = np.vstack(stacked_clusters_rep)

In [ ]:
# Define window size
window_size = 2
input_size = window_size*28 + 16*(window_size-1)
output_size = 16

In [ ]:
def extract_wav2vec_features_and_labels_from_csv(csv_path, scaler=None): #To get audio features (MFCC/Wav2Vec) of a video
    df = pd.read_csv(csv_path, header=None)
    features = df.iloc[:-1, :].values.astype(np.float32)
    video_name = csv_path.replace(".csv",'').split("/")[-1]
    if scaler is not None:
        features = scaler.transform(features)

    return features, true_descriptors[video_name]#clusters_rep_as_ground_truth_for_a_video[video_name]#true_descriptors[video_name]

def load_data_from_directory(directory_path):
    return [os.path.join(directory_path, fname) for fname in os.listdir(directory_path) if fname.endswith(".csv")]

def collate_fn(batch):
    features = [item[0] for item in batch]
    labels = [item[1] for item in batch]

    features_padded = torch.nn.utils.rnn.pad_sequence(features, batch_first=True)
    labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return features_padded, labels_padded


class SlidingWindowAudioDataset(Dataset):
    def __init__(self, video_paths, num_clusters=500, scaler=None, window_size=10):
        """
        Dataset for audio features with sliding window implementation.

        Args:
        - video_paths: List of paths to video CSVs.
        - scaler: Scaler to normalize the features.
        - window_size: Number of rows in each sliding window.
        """
        self.video_paths = video_paths
        self.scaler = scaler
        self.window_size = window_size
        self.data = []  # To store concatenated feature-label vectors
        self.labels = []  # To store the corresponding output labels (Nth label)
        self._prepare_data()

    def _prepare_data(self):
        for video_path in self.video_paths:
            features, labels = extract_wav2vec_features_and_labels_from_csv(video_path, self.scaler)

            # Normalize features using the fitted scaler
            if self.scaler:
                features = self.scaler.transform(features)

            num_rows = labels.shape[0]
            # Generate sliding windows
            for start in range(num_rows - self.window_size + 1):
                end = start + self.window_size
                # Select the current sliding window features and corresponding labels
                window_features = features[start:end]
                window_labels = labels[start:end]

                # Concatenate the first (N-1) labels with the N features for input
                # Shape: (N, 28) for features, and (N-1,) for labels
                input_vector = np.concatenate((window_features.flatten(), window_labels[:-1].flatten()))

                # The Nth label is the target/output
                target_label = window_labels[-1]
                if input_vector.shape[0] == (self.window_size*28 + output_size*(self.window_size-1)):
                  self.data.append(input_vector)
                  self.labels.append(target_label)



    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.data[idx], dtype=torch.float32),  # The concatenated feature-label vector
            torch.tensor(self.labels[idx], dtype=torch.float32),  # The output label (Nth label)
        )


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

scaler = MinMaxScaler()

mead_mfcc_path = "/content/drive/MyDrive/MEAD_MFCC/"
# Load and prepare data
dataset_path = "DatasetForLSTM/"+ Zipped_inside_folder

# Update video paths
# Go through each video name and update path of the csv --> wave to vec csvs
all_video_paths = []
for video in videos_list:
  if 'M' in video: #For MEAD
    all_video_paths.append(mead_mfcc_path+video+".csv")
  else:
    all_video_paths.append("DatasetForLSTM/"+Zipped_inside_folder+"/"+video+".csv")

train_video_paths, test_video_paths = train_test_split(all_video_paths, test_size=0.05, random_state=42)
for video_path in train_video_paths:
    features, _ = extract_wav2vec_features_and_labels_from_csv(video_path)
    scaler.partial_fit(features)



Using device: cuda


In [ ]:
#Batch Size
batch_size = 32

# Initialize the sliding window dataset
train_dataset = SlidingWindowAudioDataset(train_video_paths, scaler=scaler, window_size=window_size)
test_dataset = SlidingWindowAudioDataset(test_video_paths, scaler=scaler, window_size=window_size)

# DataLoader remains the same
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:

class MultiOutputNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        """
        Multi-output neural network that outputs an N-dimensional vector.

        Args:
        - input_dim: Number of input features.
        - output_dim: Size of the output vector (N).
        """
        super(MultiOutputNN, self).__init__()

        # Define the layers of the network
        self.fc1 = nn.Linear(input_dim, 64)  # First hidden layer
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)         # Second hidden layer
        self.fc7 = nn.Linear(32, output_dim)  # Output layer (N-dimensional vector)

    def forward(self, x):
        """
        Forward pass.

        Args:
        - x: Input tensor of shape (batch_size, input_dim)

        Returns:
        - output: Tensor of shape (batch_size, output_dim)
        """
        x = torch.relu(self.fc1(x))  # Apply ReLU activation
        x = torch.relu(self.fc2(x))  # Apply ReLU activation
        x = torch.relu(self.fc3(x))  # Apply ReLU activation
        x = torch.relu(self.fc4(x))  # Apply ReLU activation
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        output = self.fc7(x)         # Output layer (no activation)
        return output

model = MultiOutputNN(input_size, output_size).to(device)


In [ ]:
def train(model, train_loader, optimizer, device, alpha=0.5, beta=0.5):
    model.train()
    cumulative_mse_loss = 0  # MSE loss for epoch
    num_batches = len(train_loader)
    progress_bar = tqdm(train_loader, desc="Training", leave=False)

    for i, (features, labels) in enumerate(progress_bar):
        features, labels = features.to(device), labels.to(device)

        # Forward pass
        outputs = model(features)
        if outputs.shape[1] != labels.shape[1]:
          outputs = outputs[:, :-1, :]

        # Compute MSE loss
        mse_loss = F.mse_loss(outputs, labels, reduction = 'mean')

        # Backward pass and optimization
        optimizer.zero_grad()

        mse_loss.backward()
        optimizer.step()

        # Accumulate losses
        cumulative_mse_loss += mse_loss.item()

        # Update progress bar with average batch loss so far
        progress_bar.set_postfix({
            "Average MSE": cumulative_mse_loss / (i + 1)
        })

    # Calculate average loss for epoch
    avg_mse_loss = cumulative_mse_loss / num_batches

    print(f"Training Epoch Average MSE Loss: {avg_mse_loss}", sep=" ")
    return avg_mse_loss

def test(model, test_loader, device, alpha=0.5, beta=0.5):
    model.eval()
    cumulative_mse_loss = 0  # MSE loss for epoch
    num_batches = len(test_loader)
    progress_bar = tqdm(test_loader, desc="Testing", leave=False)

    with torch.no_grad():
        for i, (features, labels) in enumerate(progress_bar):
            features, labels = features.to(device), labels.to(device)

            # Forward pass
            outputs = model(features)
            if outputs.shape[1] != labels.shape[1]:
              outputs = outputs[:, :-1, :]

            # Compute MSE loss
            mse_loss = F.mse_loss(outputs, labels, reduction = 'mean')


            cumulative_mse_loss += mse_loss.item()


            # Update progress bar with average batch loss so far
            progress_bar.set_postfix({

                "Average MSE": cumulative_mse_loss / (i + 1)

            })

    # Calculate average loss for epoch
    avg_mse_loss = cumulative_mse_loss / num_batches
    print(f"Testing Epoch Average MSE Loss: {avg_mse_loss}", sep=" ")

    return avg_mse_loss


In [ ]:
# Optimizer, criterion, and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
epochs = 5

# model.load_state_dict(torch.load('drive/MyDrive/LSTM_Params/lstm_predicting_LP_descriptors.pth', weights_only=True))
model.to(device)
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train(model, train_loader, optimizer, device, alpha=1, beta=0.001)
    test_loss = test(model, test_loader, device, alpha=1, beta=0.001)

    #Step the scheduler with the test loss
    # scheduler.step(test_loss)

#     if test_loss < best_loss:
#         best_loss = test_loss
#         torch.save(model.state_dict(), 'drive/MyDrive/LSTM_Params/lstm_2_predicting_descriptors_best_model.pth')
#         print(f"New best model saved with loss: {best_loss:.2f}")

#     if epoch % 5 == 0:
#         torch.save(model.state_dict(), f'drive/MyDrive/LSTM_Params/lstm_2_predicting_descriptors_epoch{epoch}.pth')

Epoch 1/5


Training Epoch Average MSE Loss: 0.016204583308075768


Testing Epoch Average MSE Loss: 0.01260197575330854
Epoch 2/5


Training Epoch Average MSE Loss: 0.009060886052529468


Testing Epoch Average MSE Loss: 0.007379543385000355
Epoch 3/5


...

Testing Epoch Average MSE Loss: 0.008186126854080794
Epoch 4/5


Training Epoch Average MSE Loss: 0.00850004341324102


Testing Epoch Average MSE Loss: 0.007687510872267225
Epoch 5/5


Training Epoch Average MSE Loss: 0.008370268231900797


Testing Epoch Average MSE Loss: 0.008568111672772266


# Testing wiht  first (Window_Size - 1) true descriptors per sliding window



In [ ]:
#use predicted one for the next frame

# Use it if not want to train
class SlidingWindowPerVideoDatasetForAccuracy(Dataset):
    def __init__(self, video_paths, frames_to_clusters_mapping, scaler=None, window_size=10):
        """
        Dataset for generating sliding windows for an entire video.

        Args:
        - video_paths: List of paths to video CSVs.
        - scaler: Scaler to normalize the features.
        - window_size: Number of rows in each sliding window.
        """
        self.video_paths = video_paths
        self.scaler = scaler
        self.window_size = window_size
        self.frames_to_clusters_mapping = frames_to_clusters_mapping

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        features, labels = extract_wav2vec_features_and_labels_from_csv(video_path)
        video_name = video_path.replace(".csv",'').split("/")[-1]
        # Normalize features using the scaler
        if self.scaler:
            features = self.scaler.transform(features)

        num_rows = labels.shape[0]
        sliding_windows_features = []
        sliding_windows_labels = []
        #Getting ground truth for the first Window_Size - 1 frames
        for i in range(self.window_size-1):
          sliding_windows_labels.append(self.frames_to_clusters_mapping[video_name][i])
        # Generate sequential windows
        for start in range(num_rows - self.window_size + 1):
            end = start + self.window_size
            # Select the current sliding window features and corresponding labels
            window_features = features[start:end]
            window_labels = labels[start:end]

            # Concatenate the first (N-1) labels with the N features for input
            # Shape: (N, 28) for features, and (N-1,) for labels
            input_vector = np.concatenate((window_features.flatten(), window_labels[:-1].flatten()))
            sliding_windows_features.append(input_vector)
            # The Nth label is the target/output
            target_label = self.frames_to_clusters_mapping[video_name][end-1]
            sliding_windows_labels.append(target_label)

        return {
            "video": video_name,
            "features": [torch.tensor(w, dtype=torch.float32) for w in sliding_windows_features],
            "labels": [w for w in sliding_windows_labels],
        }



In [ ]:
import numpy as np
from scipy.spatial.distance import pdist, squareform

# Compute pairwise Euclidean distances
pairwise_distances = squareform(pdist(clusters_descriptors, metric='euclidean'))

# Set diagonal to infinity to ignore self-distances
np.fill_diagonal(pairwise_distances, np.inf)


# Find the indices of the top 5 nearest neighbors for each cluster
cluster_nearest_neighbors = np.argsort(pairwise_distances, axis=1)[:, :1].tolist()


# `pairwise_distances` is now a (500, 500) matrix of inter-cluster distances



In [ ]:
def sliding_window_per_video_collate_fn(batch):
    video_name = batch[0]['video']
    features = batch[0]["features"]  # List of sliding windows for each video
    labels = batch[0]["labels"]  # Corresponding labels
    return video_name, features, labels


# Initialize the sliding window per video dataset
train_dataset_ = SlidingWindowPerVideoDatasetForAccuracy(train_video_paths, frames_data, scaler=scaler, window_size=window_size)
test_dataset_ = SlidingWindowPerVideoDatasetForAccuracy(test_video_paths,frames_data, scaler=scaler, window_size=window_size)

# Initialize the data loaders
train_loader_ = DataLoader(train_dataset_, batch_size=1, shuffle=False, collate_fn=sliding_window_per_video_collate_fn)
test_loader_ = DataLoader(test_dataset_, batch_size=1, shuffle=False, collate_fn=sliding_window_per_video_collate_fn)


In [ ]:
def get_predicted_labels(outputs, cluster_descriptors, device):
    # Ensure cluster_descriptors is a torch tensor and move to device
    cluster_descriptors = torch.tensor(cluster_descriptors, device=device, dtype=outputs.dtype)
    #print(cluster_descriptors.shape)
    # Initialize an empty list to store predicted labels for each frame
    predicted_labels = []
    #outputs = outputs.squeeze(0)
    # Iterate over each frame descriptor in outputs
    for frame_descriptor in outputs:
        # Calculate Euclidean distances between the frame descriptor and each cluster descriptor
        #print(frame_descriptor.shape)

        distances = torch.norm(cluster_descriptors - frame_descriptor, dim=1)

        # Find the index of the minimum distance (i.e., closest cluster descriptor)
        predicted_label = torch.argmin(distances)
        predicted_labels.append(predicted_label.item())

    # Convert predicted labels list to a tensor on the same device
    return torch.tensor(predicted_labels, device=device)

def evaluate_video_predictions(outputs, cluster_descriptors, actual_labels_names, device, cluster_nearest_neighbors):
    # Get predicted labels based on Euclidean distance comparison
    predicted_labels = get_predicted_labels(outputs, cluster_descriptors, device)
    # Get the actual labels for the frames from frames_data and convert to torch tensor on the same device
    #print(frames_data)
    actual_labels = torch.tensor(actual_labels_names, device=device)
    # Initialize a counter for matches
    matches = 0
    total_labels = actual_labels.size(0)
    # Iterate through each label and check for matches or nearest neighbor matches
    for i in range(total_labels):
        actual_label = actual_labels[i].item()
        predicted_label = predicted_labels[i].item()

        # Check if predicted label is a match or a match with the nearest neighbor
        if predicted_label == actual_label or  predicted_label in cluster_nearest_neighbors[actual_label]:
            matches += 1

    return predicted_labels.cpu().numpy(), matches, total_labels


In [ ]:
results = {}
matched = 0
total = 0

matched_with_window = 0
total_with_window = 0
inc = window_size-1
for video_name, features_list, actual_labels in train_loader_:
    labels_for_video =[]
    for i, features in enumerate(features_list):
      features = features.to(device)
      features = features.unsqueeze(0)
      if features.shape[1] == (window_size*28 + output_size*(window_size-1)):

        outputs = model(features)  # Obtain model predictions directly in torch
        predicted_labels, matches, total_labels = evaluate_video_predictions(outputs, clusters_descriptors, [actual_labels[i+window_size-1]], device, cluster_nearest_neighbors)
        labels_for_video.extend(predicted_labels.tolist())
        matched+=matches
        total += total_labels
        matched_with_window += matches
        total_with_window +=total_labels
    matched_with_window += inc
    total_with_window += inc
    results[video_name] = labels_for_video

print("Train Accuracy without Window: ", (matched * 100)/total, "%")
print("Train Accuracy with Window: ", (matched_with_window * 100)/total_with_window, "%")

matched_test = 0
total_test = 0
matched_test_with_window = 0
total_test_with_window = 0
for video_name, features_list, actual_labels in test_loader_:
    labels_for_video =[]
    for i, features in enumerate(features_list):
      features = features.to(device)
      features = features.unsqueeze(0)
      if features.shape[1] == (window_size*28 + output_size*(window_size-1)):
        outputs = model(features)  # Obtain model predictions directly in torch

        predicted_labels, matches, total_labels = evaluate_video_predictions(outputs, clusters_descriptors, [actual_labels[i+window_size-1]], device, cluster_nearest_neighbors)
        labels_for_video.extend(predicted_labels.tolist())
        matched_test+=matches
        total_test += total_labels
        matched_test_with_window += matches
        total_test_with_window += total_labels
    matched_test_with_window += inc
    total_test_with_window += inc
    results[video_name] = labels_for_video

print("Test Accuracy without Window: ", (matched_test*100)/total_test, "%")
print("Test Accuracy with Window: ", (matched_test_with_window*100)/total_test_with_window, "%")

print("Accuracy without Window (Test + Training): ", ((matched_test+matched)*100)/(total_test+total), "%")
print("Accuracy with Window (Test + Training): ", ((matched_test_with_window+matched_with_window)*100)/(total_test_with_window+total_with_window), "%")


Train Accuracy without Window:  23.651835571143117 %
Train Accuracy with Window:  24.269241845867125 %
Test Accuracy without Window:  23.357507000789834 %
Test Accuracy with Window:  23.991953429583607 %
Accuracy without Window (Test + Training):  23.637417263675513 %
Accuracy with Window (Test + Training):  24.255655794829213 %


# Testing wiht only first (Window_Size - 1) true descriptors per video

In [ ]:
class SlidingWindowwithWindowSizeTruDescriptorsPerVideoDatasetForAccuracy(Dataset):
    def __init__(self, video_paths, frames_to_clusters_mapping, scaler=None, window_size=10):
        """
        Dataset for generating sliding windows for an entire video.

        Args:
        - video_paths: List of paths to video CSVs.
        - scaler: Scaler to normalize the features.
        - window_size: Number of rows in each sliding window.
        """
        self.video_paths = video_paths
        self.scaler = scaler
        self.window_size = window_size
        self.frames_to_clusters_mapping = frames_to_clusters_mapping

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        features, labels = extract_wav2vec_features_and_labels_from_csv(video_path)
        video_name = video_path.replace(".csv",'').split("/")[-1]
        # Normalize features using the scaler
        if self.scaler:
            features = self.scaler.transform(features)

        num_rows = labels.shape[0]
        sliding_windows_features = []
        sliding_windows_labels = []
        actual_descriptors = labels[0:self.window_size-1]

        #Getting ground truth for the first Window_Size - 1 frames
        for i in range(self.window_size-1):
          sliding_windows_labels.append(self.frames_to_clusters_mapping[video_name][i])
        # Generate sequential windows
        for start in range(num_rows - self.window_size + 1):
            end = start + self.window_size
            # Select the current sliding window features and corresponding labels
            window_features = features[start:end]


            # Concatenate the first (N-1) labels with the N features for input
            # Shape: (N, 28) for features, and (N-1,) for labels
            input_vector = window_features.flatten()
            sliding_windows_features.append(input_vector)
            # The Nth label is the target/output
            target_label = self.frames_to_clusters_mapping[video_name][end-1]
            sliding_windows_labels.append(target_label)

        return {
            "video": video_name,
            "features": [torch.tensor(w, dtype=torch.float32) for w in sliding_windows_features],
            "labels": [w for w in sliding_windows_labels],
            "descriptors": actual_descriptors,
        }



In [ ]:
def sliding_window_withWindowSizeTruDescriptors_per_video_collate_fn(batch):
    video_name = batch[0]['video']
    features = batch[0]["features"]  # List of sliding windows for each video
    labels = batch[0]["labels"]  # Corresponding labels
    actual_descriptors = batch[0]["descriptors"]
    return video_name, features, labels, actual_descriptors


# Initialize the sliding window per video dataset
train_dataset__ = SlidingWindowwithWindowSizeTruDescriptorsPerVideoDatasetForAccuracy(train_video_paths, frames_data, scaler=scaler, window_size=window_size)
test_dataset__ = SlidingWindowwithWindowSizeTruDescriptorsPerVideoDatasetForAccuracy(test_video_paths,frames_data, scaler=scaler, window_size=window_size)

# Initialize the data loaders
train_loader__ = DataLoader(train_dataset__, batch_size=1, shuffle=False, collate_fn=sliding_window_withWindowSizeTruDescriptors_per_video_collate_fn)
test_loader__ = DataLoader(test_dataset__, batch_size=1, shuffle=False, collate_fn=sliding_window_withWindowSizeTruDescriptors_per_video_collate_fn)


In [ ]:
def updateTrueDescriptorsList(actual_descriptors, window_size, new_predicted_descriptor):
    # Roll the array by 1 (shifts all rows by one, and the last row will be empty)\
    if window_size != 2:
      actual_descriptors = np.roll(actual_descriptors, shift=-1, axis=0)

      # Add the new predicted descriptor at the last row
      actual_descriptors[window_size - 2] = new_predicted_descriptor.detach().cpu().numpy()
    else:
      actual_descriptors = new_predicted_descriptor.detach().cpu().numpy()
    return actual_descriptors



In [ ]:

# MSE calculate MSE when we use predicted descriptors
train_mse = 0
for video_name, features_list, actual_labels, actual_descriptors in train_loader__:
    length = len(features_list)
    #print(length)
    predicted_descriptors = []
    #print(true_descriptors[video_name].shape)
    for i, features in enumerate(features_list):
      features = np.concatenate((features, actual_descriptors.flatten()))
      features = torch.tensor(features, dtype=torch.float32)
      features = features.to(device)
      features = features.unsqueeze(0)
      if features.shape[1] == (window_size*28 + output_size*(window_size-1)):
        outputs = model(features)
        predicted_labels, matches, total_labels = evaluate_video_predictions(outputs, clusters_descriptors, [actual_labels[i+window_size-1]], device, cluster_nearest_neighbors)
        predicted_labels = predicted_labels.tolist()
        outputs = torch.tensor(clusters_descriptors[predicted_labels[0]], dtype=torch.float32).to(device)
        actual_descriptors = updateTrueDescriptorsList(actual_descriptors, window_size, outputs)
        #print(outputs.shape)
        predicted_descriptors.append(outputs.detach().cpu().numpy())
    predicted_descriptors = np.vstack(predicted_descriptors)
    predicted_descriptors = torch.tensor(predicted_descriptors, dtype=torch.float32).to(device)
    actual_descriptors_ = true_descriptors[video_name][1:predicted_descriptors.shape[0]+1]
    actual_descriptors_ = torch.tensor(actual_descriptors_, dtype=torch.float32).to(device)
    mse_loss = F.mse_loss(predicted_descriptors, actual_descriptors_, reduction = 'mean')
    train_mse += mse_loss.item()
print("Average MSE Train:", train_mse/len(train_loader__))


# test for same
test_mse = 0
for video_name, features_list, actual_labels, actual_descriptors in test_loader__:
    length = len(features_list)
    #print(length)
    predicted_descriptors = []
    #print(true_descriptors[video_name].shape)
    for i, features in enumerate(features_list):
      features = np.concatenate((features, actual_descriptors.flatten()))
      features = torch.tensor(features, dtype=torch.float32)
      features = features.to(device)
      features = features.unsqueeze(0)
      if features.shape[1] == (window_size*28 + output_size*(window_size-1)):
        outputs = model(features)
        predicted_labels, matches, total_labels = evaluate_video_predictions(outputs, clusters_descriptors, [actual_labels[i+window_size-1]], device, cluster_nearest_neighbors)
        predicted_labels = predicted_labels.tolist()
        outputs = torch.tensor(clusters_descriptors[predicted_labels[0]], dtype=torch.float32).to(device)
        actual_descriptors = updateTrueDescriptorsList(actual_descriptors, window_size, outputs)
        #print(outputs.shape)
        predicted_descriptors.append(outputs.detach().cpu().numpy())
    predicted_descriptors = np.vstack(predicted_descriptors)
    predicted_descriptors = torch.tensor(predicted_descriptors, dtype=torch.float32).to(device)
    actual_descriptors_ = true_descriptors[video_name][1:predicted_descriptors.shape[0]+1]
    actual_descriptors_ = torch.tensor(actual_descriptors_, dtype=torch.float32).to(device)
    mse_loss = F.mse_loss(predicted_descriptors, actual_descriptors_, reduction = 'mean')
    test_mse += mse_loss.item()
print("Average MSE Test:", test_mse/len(test_loader__))

Average MSE Train: 0.331453685944762
Average MSE Test: 0.24782005577918984


In [ ]:

# Give accuracy instead of MSE when we use predicted descriptors
results = {}
matched = 0
total = 0

matched_with_window = 0
total_with_window = 0
inc = window_size-1
for video_name, features_list, actual_labels, actual_descriptors in train_loader__:
    labels_for_video =[]
    for i, features in enumerate(features_list):
      features = np.concatenate((features, actual_descriptors.flatten()))
      features = torch.tensor(features, dtype=torch.float32)
      features = features.to(device)
      features = features.unsqueeze(0)
      if features.shape[1] == (window_size*28 + output_size*(window_size-1)):
        outputs = model(features)  # Obtain model predictions directly in torch
        predicted_labels, matches, total_labels = evaluate_video_predictions(outputs, clusters_descriptors, [actual_labels[i+window_size-1]], device, cluster_nearest_neighbors)
        predicted_labels = predicted_labels.tolist()
        outputs = torch.tensor(clusters_descriptors[predicted_labels[0]], dtype=torch.float32).to(device)
        actual_descriptors = updateTrueDescriptorsList(actual_descriptors, window_size, outputs)
        labels_for_video.extend(predicted_labels)
        matched+=matches
        total += total_labels
        matched_with_window += matches
        total_with_window +=total_labels
    matched_with_window += inc
    total_with_window += inc
    results[video_name] = labels_for_video

print("Train Accuracy without Window: ", (matched * 100)/total, "%")
print("Train Accuracy with Window: ", (matched_with_window * 100)/total_with_window, "%")

matched_test = 0
total_test = 0
matched_test_with_window = 0
total_test_with_window = 0
for video_name, features_list, actual_labels, actual_descriptors in test_loader__:
    labels_for_video =[]
    for i, features in enumerate(features_list):
      features = np.concatenate((features, actual_descriptors.flatten()))
      features = torch.tensor(features, dtype=torch.float32)
      features = features.to(device)
      features = features.unsqueeze(0)
      if features.shape[1] == (window_size*28 + output_size*(window_size-1)):
        outputs = model(features)  # Obtain model predictions directly in torch
        predicted_labels, matches, total_labels = evaluate_video_predictions(outputs, clusters_descriptors, [actual_labels[i+window_size-1]], device, cluster_nearest_neighbors)
        predicted_labels = predicted_labels.tolist()
        outputs = torch.tensor(clusters_descriptors[predicted_labels[0]], dtype=torch.float32).to(device)
        actual_descriptors = updateTrueDescriptorsList(actual_descriptors, window_size, outputs)
        labels_for_video.extend(predicted_labels)
        matched_test+=matches
        total_test += total_labels
        matched_test_with_window += matches
        total_test_with_window += total_labels
    matched_test_with_window += inc
    total_test_with_window += inc
    results[video_name] = labels_for_video

print("Test Accuracy without Window: ", (matched_test*100)/total_test, "%")
print("Test Accuracy with Window: ", (matched_test_with_window*100)/total_test_with_window, "%")

print("Accuracy without Window (Test + Training): ", ((matched_test+matched)*100)/(total_test+total), "%")
print("Accuracy with Window (Test + Training): ", ((matched_test_with_window+matched_with_window)*100)/(total_test_with_window+total_with_window), "%")


Train Accuracy without Window:  42.847407601926605 %
Train Accuracy with Window:  43.3095846487133 %
Test Accuracy without Window:  43.18230774754075 %
Test Accuracy with Window:  43.65264450892778 %
Accuracy without Window (Test + Training):  42.863813393809515 %
Accuracy with Window (Test + Training):  43.326393244010575 %
